In [ ]:
import json
import re
import requests
import urllib

from IPython.core.display import HTML

## Settings

In [ ]:
# Selected reserve
reserve  = "Settler Creeks"

# Login info for www.thehunter.com
username = "<email>"
password = "<password>"

## Scrape Data from www.thehunter.com

In [ ]:
session = requests.Session()

data  = urllib.parse.urlencode({"email": username, "password": password})
r1    = session.post("https://www.thehunter.com/login?xhr=true&return_to=/",data=data)
r2    = session.get ("https://www.thehunter.com/#feed"         
                    )
token = re.search(r'var userAccessToken = "(\w+)";',r2.text)[1]
oauth = re.search(r'oauth_consumer_key: "(\w+)"',   r2.text)[1]

data  = urllib.parse.urlencode({"oauth_access_token": token})
r3    = session.post("https://api.thehunter.com/v1/Mission/missions",       data=data)
r4    = session.post("https://api.thehunter.com/v1/Me/me",                  data=data)

data  = urllib.parse.urlencode({"oauth_consumer_key": oauth})
r5    = session.post("https://api.thehunter.com/v1/Application/application",data=data)

In [ ]:
activeMissions = json.loads(r3._content)["active"]
myItems        = dict((int(iid),cnt) for iid,cnt in json.loads(r4._content)["items"].items())
application    = json.loads(r5._content)
missions       = dict([(m["id"],m) for m in application["missions"]])
species        = dict([(s["id"],s) for s in application["species" ]])
reserves       = dict([(r["id"],r) for r in application["reserves"]])
items          = dict([(i["id"],i) for i in application["items"   ]])

In [ ]:
# add mission group to the individual missions
missionGroups = {}
for mg in application["mission_groups"]:
    mgid = mg["id"]
    missionGroups[mgid] = mg
    for mid in mg["missions"]:
        missions[mid]["missionGroup"] = mgid

## Filter Missions by Keywords

Check whether at least one of the keywords appears in one of the titles.

In [ ]:
def checkKeywords(kws,titles):
    for ti in titles:
        pattern = f"({'|'.join(re.escape(kw) for kw in kws)})"
        newTi,n = re.subn(pattern,r"<mark>\1</mark>",ti,flags=re.IGNORECASE)
        if n > 0:
            return newTi
    return None

Extract keywords from a reserve. Whitehart Island is sometimes referred to just as Whitehart and the apostrophe in Logger's Point is used inconsistently.

In [ ]:
def reserveKeyword(r):
    nm = r["name"]
    if nm == "Whitehart Island":
        return ("Whitehart", nm)
    elif nm == "Logger's Point":
        return ("Loggers Point", nm)
    return (nm,)

Extract keywords from a species. Use only Typical variants and delete Non-Typical (e.g. for Mule and *tail deer).

In [ ]:
def speciesKeyword(s):
    nm = s["name"]
    if nm.endswith(" (Typical)"):
        nm = nm[:-10]
    elif nm.endswith(" (Non-Typical)"):
        return tuple()
    return (nm,)

Extract keywords for an item

In [ ]:
def itemKeyword(i):
    nm  = i["name"]
    snm = i["shortname"]
    if nm == "Compound Bow \"Parker Python\"":
        return ("Parker Python Compound Bow",nm,snm)
    elif nm.startswith("Aimpoint"):
        return (nm,"Aimpoint Sight")
    elif snm is not None:
        return (nm,snm)
    else:
        return (nm,)        

Collect all bad keywords

In [ ]:
badKeywords  = set()
goodKeywords = set()

# get id of selected reserve
rid = [rid for rid,r in reserves.items() if r["name"] == reserve][0]

goodKeywords.add(reserve)

# add keywords for all other reserves
for orid,r in reserves.items():
    if rid != orid:
        badKeywords.update(reserveKeyword(r))
     
# add keywords for the species in the selected reserve
for sid in reserves[rid]["species"]:
    goodKeywords.update(speciesKeyword(species[sid]))
    
# add keywords for all species that are not in the selected reserve
for sid,s in species.items():
    if sid not in reserves[rid]["species"]:
        badKeywords.update(speciesKeyword(s))

# add keywords for all my items
for iid,count in myItems.items():
    if count[0] > 0:
        goodKeywords.update(itemKeyword(items[iid]))

# add keywords for all items that I do not own
for iid,i in items.items():
    if iid not in myItems.keys() or myItems[iid][0] == 0:
        badKeywords.update(itemKeyword(i))

In [ ]:
# Remove good keywords from bad keywords
# e.g. if the selected reserve has a Willow Ptarmigan resulting in a good keyword "Ptarmigan"
# and the bad keywords contain "Ptarmigan" because of the "Rock Ptarmigan", then a mission which
# mentions just a Ptarmigan shall not be filtered out

badKeywords -= goodKeywords

Filter missions that contain no bad keywords

In [ ]:
goodMissions = []
badMissions  = []

for mid in activeMissions:
    m  = missions[mid]
    mg = missionGroups[m["missionGroup"]]
    
    mgTitle = mg["title"]
    mTitle  = m["title"]
    tTitles = tuple(obj["title"] for obj in m["objectives"])
    
    ret = checkKeywords(badKeywords, (mgTitle, mTitle) + tTitles)
    if ret is not None:
        badMissions.append((m,mg,ret))
    else:
        goodMissions.append((m,mg))

## Output HTML Table with Filtered Missions
Missions that are ready to be completed are shown on top. After that all the blocked missions appear together with the title and highlighted keyword, which caused the mission to be filtered out.

In [ ]:
out = "<table>"

for m,mg in goodMissions:
    out += f"<tr><td>{m['title']}</td><td>{mg['title']}</td><td style='text-align:left'><ul>"
    for obj in m["objectives"]:
        out += f"<li>{obj['title']}</li>"
    out += "</ul></td></tr>"
    
for m,mg,st in badMissions:
    out += f"<tr><td>{m['title']}</td><td>{mg['title']}</td><td style='text-align:left'>{st}</td></tr>"
    
out += "</table>"

HTML(out)